# Z3
参考：<https://blog.shi1011.cn/learn/1789>
教程：<https://zhuanlan.zhihu.com/p/30548907>

In [1]:
from z3 import *

x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7) 

[y = 0, x = 7]


## DEFCAMP 2017 Misc 题 forgot my key

题目如下：

I forgot my flag & key. Help me recover them.

5616f5962674d26741d2810600a6c5647620c4e3d2870177f09716b2379012c342d3b584c5672195d653722443f1c39254360007010381b721c741a532b03504d2849382d375c0d6806251a2946335a67365020100f160f17640c6a05583f49645d3b557856221b2


```php
function my_encrypt($flag, $key) {
  $key = md5($key);
  $message = $flag . "|" . $key;

  $encrypted = chr(rand(0, 126));
  for($i=0;$i<strlen($message);$i++) {
    $encrypted .= chr((ord($message[$i]) + ord($key[$i % strlen($key)]) + ord($encrypted[$i])) % 126);
  }
  $hexstr = unpack('h*', $encrypted);
  return array_shift($hexstr);
}
```
encrypted[i+1] = (message[i]+ key[i % klen] + encrypted[i]) % 126  
已知：encrypted（除第一位外），message倒数第 33 位，因此可以从倒数第33位一个个推。  

In [2]:
from z3 import *
import binascii

s = '5616f5962674d26741d2810600a6c5647620c4e3d2870177f09716b2379012c342d3b584c5672195d653722443f1c39254360007010381b721c741a532b03504d2849382d375c0d6806251a2946335a67365020100f160f17640c6a05583f49645d3b557856221b2'

encrypted = []
for i in range(0, len(s), 2):
    encrypted.append(binascii.unhexlify(s[i+1] + s[i])[0])

print('message len:', len(encrypted)-1)
print(encrypted)
# 声明变量，encrypted 是已知，因此 IntVal 即可
encrypted = [IntVal(i) for i in encrypted]
message = [Int('flag%d' % i) for i in range(len(encrypted)-1)]
# 创建一个求解器，求解全靠它
solver = Solver()

ml = len(encrypted) - 1

# 添加明文字符的约束条件
for i in range(ml):
    if i == ml - 33:
        solver.add(message[i] == ord('|'))
    else:
        # 肯定是可见字符，因此限定范围如下
        solver.add(message[i] < 127)
        solver.add(message[i] >= 32)
# 添加明文和密文对照关系的约束条件
for i in range(ml):
    solver.add(encrypted[i+1] == (message[i] + message[ml-32+i%32] + encrypted[i]) % 126)

if solver.check() == sat:
    m = solver.model()
    s = []
    for i in range(ml):
        s.append(m[message[i]].as_long())
    print(bytes(s))
else:
    print('unsat') 

message len: 103
[101, 97, 95, 105, 98, 71, 45, 118, 20, 45, 24, 96, 0, 106, 92, 70, 103, 2, 76, 62, 45, 120, 16, 119, 15, 121, 97, 43, 115, 9, 33, 60, 36, 61, 91, 72, 92, 118, 18, 89, 109, 53, 39, 66, 52, 31, 60, 41, 69, 99, 0, 112, 16, 48, 24, 123, 18, 124, 20, 90, 35, 11, 83, 64, 45, 72, 57, 40, 61, 87, 12, 109, 8, 38, 21, 42, 73, 54, 83, 106, 55, 86, 32, 16, 0, 31, 6, 31, 103, 4, 108, 10, 85, 56, 79, 105, 84, 61, 91, 117, 88, 38, 18, 43]
b'DCTF{0d940de38493d96dc6255cbb2c2ac7a2db1a7792c74859e95215caa6b57c69b2}|6941f4cac9b7784fdd77e11b51cd0d64'


In [2]:
from z3 import *
from functools import reduce
enc = "kcGXlcG9ihRqlYy5"
maps = StringVal("3GHIJKLMNOPQRSTUb=cdefghijklmnopWXYZ/12+406789VaqrstuvwxyzABCDEF5")
sol = reduce(Concat, [Int2BV(If(i == '=', 0, IndexOf(maps, i, 0)), 6) for i in enc])
arr = reversed([chr(simplify(Extract(i + 7, i, sol)).as_long()) for i in range(0, sol.size(), 8)])
print("".join(arr))

i am mas0n. 


## hgame2019 わかります (useZ3)

```c
s1 = new_int_arr(36);
s2 = new_int_arr(36);
for ( i = 0; i < v5; ++i )
{
    s1[i] = a1[i] >> 4;
    s2[i] = a1[i] & 0xF;
}
s3 = for3(s1, magic, 6);
s4 = for2(s2, magic, 6);
for ( j = 0; j <= 35; ++j )
{
    if ( s3[j] != final1[j] || s4[j] != final2[j] )
        ok = 0;
}
```

```c
_DWORD *__fastcall for3(int *s, int *magic, int len)
{
  int i; // [rsp+2Ch] [rbp-14h]
  int j; // [rsp+30h] [rbp-10h]
  int k; // [rsp+34h] [rbp-Ch]
  _DWORD *v8; // [rsp+38h] [rbp-8h]

  v8 = new_int_arr(len);
  for ( i = 0; i < len; ++i )
  {
    for ( j = 0; j < len; ++j )
    {
      for ( k = 0; k < len; ++k )
        v8[len * i + j] += s[len * i + k] * magic[len * k + j];
    }
  }
  return v8;
}

_DWORD *__fastcall for2(int *s, int *magic, int len)
{
  int i; // [rsp+20h] [rbp-10h]
  int j; // [rsp+24h] [rbp-Ch]
  int *v7; // [rsp+28h] [rbp-8h]

  v7 = new_int_arr(len);
  for ( i = 0; i < len; ++i )
  {
    for ( j = 0; j < len; ++j )
      v7[len * i + j] = s[len * i + j] + magic[len * i + j];
  }
  return v7;
}
```


In [12]:
from z3 import *
arr1 = [0X7A,0XCF,0X8C,0X95,0X8E,0XA8,0X5F,0XC9,0X7A,0X91,0X88,0XA7,0X70,0XC0,0X7F,0X89,0X86,0X93,0X5F,0XCF,0X6E,0X86,
        0X85,0XAD,0X88,0XD4,0XA0,0XA2,0X98,0XB3,0X79,0XC1,0X7E,0X7E,0X77,0X93]
arr2 = [0x10,0x08,0X08,0x0E,0X06,0X0B,0X5,0X17,0X05,0X0A,0X0C,0X17,0X0E,0X17,0X13,0X07,0X08,0X0A,0X04,0X0D,0X16,0X11,0X0B,
        0X16,0X06,0X0E,0X02,0X0B,0X12,0X09,0X05,0X08,0X08,0X0A,0X10,0X0D]
arr3 = [8,1,7,1,1,0,4,8,1,2,3,9,3,8,6,6,4,8,3,5,7,8,8,7,0,9,0,2,3,4,2,3,2,5,4,0]

# arr1对应2120,arr2对应21c0,arr3对应2080

input_flag = [BitVec("%d" % i, 8) for i in range(36)]  # 初始化序列
s = Solver()  # 创建约束求解器
for i in range(36):
    s.add(input_flag[i] < 127)  # 添加约束条件①
    s.add(input_flag[i] >= 32)
# 接下来还原代码
s1 = [0] * 36
s2 = [0] * 36  # 最开始的两个变换之后的数组
s3 = [0] * 36
s4 = [0] * 36  # 这两个用来存放对应的两个函数的结果
for i in range(36):
    s1[i] = input_flag[i] >> 4
    s2[i] = input_flag[i] & 15
for i in range(6):
    for j in range(6):
        for k in range(6):
            s3[6 * i + j] += s1[6 * i + k] * arr3[6 * k + j]

for i in range(6):
    for j in range(6):
        s4[6 * i + j] = s2[6 * i + j] + arr3[6 * i + j]
# 代码还原结束
for i in range(36):  # 添加约束条件②
    s.add(s3[i] == arr1[i])
    s.add(s4[i] == arr2[i])
flag = []
strflag = ""
if s.check() == sat:  # 检测是否有解
    result = s.model()
    for i in input_flag:  # 因为最后得出的是等式，先遍历temp，把temp的每个依次赋给i
        flag.append(result[i])  # 然后找到每个temp对应的解,附加到空列表的后面
else:
    print("无解")


''.join(chr(flag[i].as_long()) for i in range(36))

'hgame{1_think_Matr1x_is_very_usef5l}'